# Efficiency
https://distributed.readthedocs.io/en/latest/efficiency.html

In [1]:
import numpy as np
from dask.distributed import Client

In [2]:
client = Client()
client

Client Scheduler: tcp://127.0.0.1:40355 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 2 Cores: 2 Memory: 8.28 GB


## Leave data on the cluster

In [3]:
x = client.submit(np.random.random, (1000, 1000))
type(x)

distributed.client.Future

In [4]:
%time x.result().shape # Slow from lots of data transfer

CPU times: user 25.3 ms, sys: 23.4 ms, total: 48.8 ms
Wall time: 55.7 ms


(1000, 1000)

In [5]:
%time client.submit(lambda a: a.shape, x).result()  # fast

CPU times: user 5.77 ms, sys: 7.45 ms, total: 13.2 ms
Wall time: 21.8 ms


(1000, 1000)

## Use larger tasks

Slow
```python
>>> futures = client.map(f, seq)
>>> len(futures)  # avoid large numbers of futures
1000000000
```
Fast

```python
>>> def f_many(chunk):
...     return [f(x) for x in chunk]

>>> from toolz import partition_all
>>> chunks = partition_all(1000000, seq)  # Collect into groups of size 1000

>>> futures = client.map(f_many, chunks)
>>> len(futures)  # Compute on larger pieces of your data at once
1000
```

## Adjust between Threads and Processes
By default a single Worker runs many computations in parallel using as many threads as your compute node has cores. When using pure Python functions this may not be optimal and you may instead want to run several separate worker processes on each node, each using one thread. When configuring your cluster you may want to use the options to the dask-worker executable as follows:
```
$ dask-worker ip:port --nprocs 8 --nthreads 1
```
Note that if you’re primarily using NumPy, Pandas, SciPy, Scikit Learn, Numba, or other C/Fortran/LLVM/Cython-accelerated libraries then this is not an issue for you. Your code is likely optimal for use with multi-threading.